In [1]:
import os
import pdb
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import random
import ipdb

print(f'Can I can use GPU now? -- {torch.cuda.is_available()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Can I can use GPU now? -- True


Load data from the KITTI dataset and perform train-test split:


In [2]:
class KITTIDataset(Dataset):
    def __init__(self, root_dir):
        """
        Args:
            root_dir (string): Directory with all the point clouds.
        """
        self.root_dir = root_dir
        self.files = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        point_cloud_path = os.path.join(self.root_dir, self.files[idx])
        point_cloud = self.load_point_cloud_from_bin(point_cloud_path)
        return point_cloud
    
    def load_point_cloud_from_bin(self, bin_path):
        with open(bin_path, 'rb') as f:
            content = f.read()
            point_cloud = np.frombuffer(content, dtype=np.float32)
            point_cloud = point_cloud.reshape(-1, 4)  # KITTI point clouds are (x, y, z, intensity)
        return torch.from_numpy(point_cloud)
        

train_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/training/velodyne_reduced'
test_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/testing/velodyne_reduced'

train_set = KITTIDataset(root_dir=train_dir)
test_set = KITTIDataset(root_dir=test_dir)
        
        
#batched_train_set = DataLoader(train_set, batch_size=4, shuffle=False)
#batched_test_set = DataLoader(test_set, batch_size=4, shuffle=False)


In [6]:
class PillarFeatureNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(PillarFeatureNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        
        self.to(device)

    def forward(self, x):
        # Input x is of shape (D, P, N)
        # Convert it to (P, D, N) for 1x1 convolution      
        x = x.to(device)
        x = x.permute(1, 0, 2)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # Max pooling operation over the points' dimension
        x, _ = torch.max(x, dim=2)  # Output shape: (P, C)
        return x.T  # Output shape: (C, P)



class Pillarization:
    def __init__(self, x_min, x_max, y_min, y_max, z_min, z_max, pillar_size, max_points_per_pillar, aug_dim):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.z_min = z_min
        self.z_max = z_max
        self.pillar_size = pillar_size
        self.max_points_per_pillar = max_points_per_pillar
        self.aug_dim = aug_dim
        self.num_x_pillars = int((self.x_max - self.x_min) / self.pillar_size[0])
        self.num_y_pillars = int((self.y_max - self.y_min) / self.pillar_size[1])
        

    def make_pillars(self, points):
        """
        Convert point cloud (x, y, z) into pillars.
        """
        # Mask points outside of our defined boundaries
        
        mask = (
            (points[:, 0] >= self.x_min) & (points[:, 0] <= self.x_max) &
            (points[:, 1] >= self.y_min) & (points[:, 1] <= self.y_max) &
            (points[:, 2] >= self.z_min) & (points[:, 2] <= self.z_max)
        )
        points = points[mask]

        
        # Using numpy's digitize to find the interval/bin each point belongs to.
        self.x_indices = torch.tensor(np.digitize(points[:, 0],
            np.linspace(self.x_min, self.x_max, self.num_x_pillars))) - 1
        
        self.y_indices = torch.tensor(np.digitize(points[:, 1], 
            np.linspace(self.y_min, self.y_max, self.num_y_pillars))) - 1
        
        pillars = torch.zeros((self.num_x_pillars, self.num_y_pillars, self.max_points_per_pillar, 
            self.aug_dim))
        
        # Count how many points are in each pillar to ensure we don't exceed `max_points_per_pillar`
        count = torch.zeros((self.num_x_pillars, self.num_y_pillars), dtype=torch.long)
        
        if (device != torch.device('cpu')):
            self.x_indices.to(device)
            self.y_indices.to(device)
            points = points.to(device)
            pillars = pillars.to(device)
            count = count.to(device)
           
        # TODO: Calculate pillar x-y center:
        pillar_x_center = self.x_indices * self.pillar_size[0] + self.pillar_size[0] / 2.0
        pillar_y_center = self.y_indices * self.pillar_size[1] + self.pillar_size[1] / 2.0 

        
        # TODO: Store points in the pillars in a vectorized way filling the pillars tensor:        
        for i in range(points.shape[0]):
            x_ind = self.x_indices[i]
            y_ind = self.y_indices[i]
            
            if count[x_ind, y_ind] < self.max_points_per_pillar:
                # Compute x_c, y_c and z_c
                x_c = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0) - points[i, 0]
                y_c = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0) - points[i, 1]
                z_c = (self.z_min + self.z_max) / 2 - points[i, 2] # assuming the z-center is the midpoint
                
                # Calculate pillar center
                x_pillar_center = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0)
                y_pillar_center = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0)
                
                # Add original x, y, and z coordinates, then x_c, y_c, z_c
                pillars[x_ind, y_ind, count[x_ind, y_ind], :3] = points[i, :3]
                
                if (device != torch.device('cpu')): 
                    pillars[x_ind, y_ind, count[x_ind, y_ind], 3:6] = torch.tensor([x_c, y_c, z_c]).to(device)
                else: 
                    pillars[x_ind, y_ind, count[x_ind, y_ind], 3:6] = torch.tensor([x_c, y_c, z_c])
                    
                pillars[x_ind, y_ind, count[x_ind, y_ind], 6] = x_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 0]
                pillars[x_ind, y_ind, count[x_ind, y_ind], 7] = y_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 1]
                
                count[x_ind, y_ind] += 1
                
        
        # Zero-padding if too few point, random sampling if too many points:
        for i in range(self.num_x_pillars):
            for j in range(self.num_y_pillars):
                if pillars[i, j].shape[0] > self.max_points_per_pillar:
                    # Randomly sample points if there are too many for a given pillar
                    pillars[i, j] = pillars[i, j][torch.randperm(pillars[i, j].shape[0])[:self.max_points_per_pillar]]
                elif pillars[i, j].shape[0] < self.max_points_per_pillar:
                    # Zero pad if there are too few points for a given pillar
                    pillars[i, j] = torch.cat((pillars[i, j], torch.zeros((self.max_points_per_pillar - pillars[i, j].shape[0], pillars[i, j].shape[1]))))
        
        # Reshape pillars to size (D,P,N):
        pillars = pillars.permute(3, 0, 1, 2).reshape(D, -1, N)
        
        return pillars


class PseudoImageDataset(Dataset):
    def __init__(self, train_dir, D, N, transform=None):
        self.train_dir = train_dir
        self.filenames = [f for f in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, f))]
        self.transform = transform
        
        self.pillarizer = Pillarization(aug_dim=D, x_min=-40.0, x_max=40.0, y_min=-25.0, y_max=25.0, 
                                        z_min=-3, z_max=3, pillar_size=(0.5, 0.5), max_points_per_pillar=N)
        self.feature_extractor = PillarFeatureNet(D, 64)

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        point_cloud = train_set.load_point_cloud_from_bin(os.path.join(self.train_dir, self.filenames[idx]))
        pillars = self.pillarizer.make_pillars(point_cloud)
        
        # Apply linear activation, batchnorm, and ReLU for feature extraction from pillars tensor
        features = self.feature_extractor(pillars)
        
        # Generate pseudo-image (GPU)
        pseudo_image = torch.zeros(features.shape[0], self.pillarizer.num_y_pillars, self.pillarizer.num_x_pillars).to(device)
        
        # Scatter the features back to their original pillar locations
        print(f'Loading point cloud number {idx}')
        for i in range(features.shape[1]):
            ipdb.set_trace()
            x_ind = self.pillarizer.x_indices[i].long() # SUS: Is the indexing here correct?
            y_ind = self.pillarizer.y_indices[i].long()
            pseudo_image[:, y_ind, x_ind] = features[:, i]

        
        if self.transform:
            pseudo_image = self.transform(pseudo_image)
            
        return pseudo_image
        
        
# Create the dataset and DataLoader
D = 9
N = 100
dataset = PseudoImageDataset(train_dir=train_dir, D=D, N=N)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

for batch_idx, (images) in enumerate(train_loader):
    print(batch_idx)

Loading point cloud number 4591
> /tmp/ipykernel_597539/1872366522.py(150)__getitem__()
    149             ipdb.set_trace()
--> 150             x_ind = self.pillarizer.x_indices[i].long() # SUS: Is the indexing here correct?
    151             y_ind = self.pillarizer.y_indices[i].long()

ipdb> len(self.pillarizer.x_indices)
19804
ipdb> features.size()
torch.Size([64, 16000])
ipdb> features.shape[1]
16000
ipdb> self.pillarizr.x_indices.size()
*** AttributeError: 'PseudoImageDataset' object has no attribute 'pillarizr'
ipdb> self.pillarizer.x_indices.size()
torch.Size([19804])
ipdb> self.x_indices.size()
*** AttributeError: 'PseudoImageDataset' object has no attribute 'x_indices'
ipdb> exit


In [ ]:
class KITTIDataset(Dataset):
    def __init__(self, root_dir, debug=False):
        """
        Args:
            root_dir (string): Directory with all the point clouds.
        """
        self.root_dir = root_dir
        self.files = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]
            
        if debug:
            random_sample = random.randint(1, 7400)
            self.files = [self.files[random_sample]]
        

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        point_cloud_path = os.path.join(self.root_dir, self.files[idx])
        point_cloud = self.load_point_cloud_from_bin(idx, point_cloud_path)  # Assuming point clouds are stored as .pt files
        return point_cloud
    
    def load_point_cloud_from_bin(self, idx, root_dir):
        idx_as_string = str(idx)
        shift = len(idx_as_string)
        file = '000000'
        for i in range(len(file) - shift):
            file[i] = idx_as_string[i-len(file)]
                    
        bin_path = os.path.join(root_dir, )
        with open(bin_path, 'rb') as f:
            content = f.read()
            point_cloud = np.frombuffer(content, dtype=np.float32)
            point_cloud = point_cloud.reshape(-1, 4)  # KITTI point clouds are (x, y, z, intensity)
        return torch.from_numpy(point_cloud)
        

train_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/training/velodyne_reduced'
test_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/testing/velodyne_reduced'
# TODO: Add label dir

train_set = KITTIDataset(root_dir=train_dir, debug=False)
test_set = KITTIDataset(root_dir=test_dir, debug=False)

batched_train_set = DataLoader(train_set, batch_size=4, shuffle=False)
batched_test_set = DataLoader(test_set, batch_size=4, shuffle=False)


In [ ]:
class PillarFeatureNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(PillarFeatureNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Input x is of shape (D, P, N)
        # Convert it to (P, D, N) for 1x1 convolution
        x = x.permute(1, 0, 2)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # Max pooling operation over the points' dimension
        x, _ = torch.max(x, dim=2)  # Output shape: (P, C)
        return x.T  # Output shape: (C, P)



class Pillarization:
    def __init__(self, x_min, x_max, y_min, y_max, z_min, z_max, pillar_size, max_points_per_pillar, aug_dim):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.z_min = z_min
        self.z_max = z_max
        self.pillar_size = pillar_size
        self.max_points_per_pillar = max_points_per_pillar
        self.aug_dim = aug_dim
        self.pillars_dict = {} # DEPRECATE?
        self.num_x_pillars = int((self.x_max - self.x_min) / self.pillar_size[0])
        self.num_y_pillars = int((self.y_max - self.y_min) / self.pillar_size[1])
        

    def make_pillars(self, points):
        """
        Convert point cloud (x, y, z) into pillars.
        """
        # Mask points outside of our defined boundaries
        
        mask = (
            (points[:, 0] >= self.x_min) & (points[:, 0] <= self.x_max) &
            (points[:, 1] >= self.y_min) & (points[:, 1] <= self.y_max) &
            (points[:, 2] >= self.z_min) & (points[:, 2] <= self.z_max)
        )
        points = points[mask]

        
        # Using numpy's digitize to find the interval/bin each point belongs to.
        # TODO: Get rid of these unnecessary copies of tensors
        self.x_indices = torch.tensor(np.digitize(points[:, 0], np.linspace(self.x_min, self.x_max, self.num_x_pillars))) - 1
        self.y_indices = torch.tensor(np.digitize(points[:, 1], np.linspace(self.y_min, self.y_max, self.num_y_pillars))) - 1

        # TODO: Calculate pillar x-y center:
        pillar_x_center = self.x_indices * self.pillar_size[0] + self.pillar_size[0] / 2.0
        pillar_y_center = self.y_indices * self.pillar_size[1] + self.pillar_size[1] / 2.0 

        pillars = torch.zeros((self.num_x_pillars, self.num_y_pillars, self.max_points_per_pillar, self.aug_dim))
        
        # Count how many points are in each pillar to ensure we don't exceed `max_points_per_pillar`
        count = torch.zeros((self.num_x_pillars, self.num_y_pillars), dtype=torch.long)
        
        # TODO: Store points in the pillars in a vectorized way filling the pillars tensor:        
        for i in range(points.shape[0]):
            x_ind = self.x_indices[i]
            y_ind = self.y_indices[i]
            
            if count[x_ind, y_ind] < self.max_points_per_pillar:
                # Compute x_c, y_c and z_c
                x_c = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0) - points[i, 0]
                y_c = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0) - points[i, 1]
                z_c = (self.z_min + self.z_max) / 2 - points[i, 2] # assuming the z-center is the midpoint
                
                # Calculate pillar center
                x_pillar_center = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0)
                y_pillar_center = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0)
                
                # Add original x, y, and z coordinates, then x_c, y_c, z_c
                pillars[x_ind, y_ind, count[x_ind, y_ind], :3] = points[i, :3]
                pillars[x_ind, y_ind, count[x_ind, y_ind], 3:6] = torch.tensor([x_c, y_c, z_c]) # Sus
                pillars[x_ind, y_ind, count[x_ind, y_ind], 6] = x_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 0]
                pillars[x_ind, y_ind, count[x_ind, y_ind], 7] = y_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 1]
                
                count[x_ind, y_ind] += 1
        
        # Zero-padding if too few point, random sampling if too many points:
        for i in range(self.num_x_pillars):
            for j in range(self.num_y_pillars):
                if pillars[i, j].shape[0] > self.max_points_per_pillar:
                    # Randomly sample points if there are too many for a given pillar
                    pillars[i, j] = pillars[i, j][torch.randperm(pillars[i, j].shape[0])[:self.max_points_per_pillar]]
                elif pillars[i, j].shape[0] < self.max_points_per_pillar:
                    # Zero pad if there are too few points for a given pillar
                    pillars[i, j] = torch.cat((pillars[i, j], torch.zeros((self.max_points_per_pillar - pillars[i, j].shape[0], pillars[i, j].shape[1]))))
        
        # Reshape pillars to size (D,P,N):
        pillars = pillars.permute(3, 0, 1, 2).reshape(D, -1, N)
        
        return pillars

